In [1]:
import numpy as np
import pandas as pd
import re
import os
from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

# # ignore "SettingWithCopyWarning"
# pd.set_option("mode.chained_assignment", None)

In [8]:
def load_data() -> pd.DataFrame:
    """
    ====================================================
    Load data from a directory and covert into a single dataframe.
    """
    # load the csv files in the 'data' directory and save it in a list
    files = [file for file in os.listdir('data/')]
    all_df = pd.DataFrame()  # empty dataframe
    for file in files:
        df = pd.read_csv(f'data/{file}')   # read each file 
        all_df = pd.concat([all_df, df], axis='index')  # concatenate each file
    # filter out duplicated records
    all_df = all_df[all_df.duplicated(keep='first')]
    return all_df 


class PreProcessData(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
    
    def transform(self, data: pd.DataFrame):
        """
        ====================================================
        Clean and tranform the features.
        """
        
        df = data.copy()
        ### drop the missing values
        df = df.dropna()
        # clean the data
        for col in df.columns:
            if col == 'address':
                df[col] = df[col].apply(self.get_address)
            elif col != 'title':
                df[col] = df[col].apply(self.clean_text)
                
        # convert to numeric data type
        for col in ['bed', 'bath', 'toilet', 'pkn_space', 'price']:
            df[col] = pd.to_numeric(df[col])

        df = df.drop(columns=['title'])
        # select houses with bedrooms between 2 and 7
        df = df.loc[(df['bed'] > 1) & (df['toilet'] < 8)]
        # select houses with bathrooms between 2 and 7
        df = df.loc[(df['bath'] > 1) & (df['bath'] < 8)]
        # select houses with pkn_space between 1 and 10
        df = df.loc[(df['pkn_space'] > 2) & (df['pkn_space'] < 11)]
        #  outliers for price
        cut_off = np.percentile(df['price'], 97)  # remove prices above the 97th percentile
        df = df.loc[df['price'] <= cut_off]      
        # fill the missing values in 'pkn_space' with the median value
        median = df['pkn_space'].median()
        df['pkn_space'] = np.where(pd.isna(df['pkn_space']), median, df['pkn_space']) 

        # rename column
        df = df.rename(columns={'address': 'location'}) 
        locations = ['Lekki','Ajah','Ikeja','Ikoyi','Victoria Island (VI)','Ibeju Lekki','Isheri North','Magodo','Maryland',
             'Ikorodu','Surulere','Ojodu','Ipaja','Ikotun','Isolo']

        # Filter out locations with fewer counts
        df = df.loc[df['location'].isin(locations)]
        le_location = LabelEncoder()
        df['location'] = le_location.fit_transform(df['location'])
        # transform the price
        df['log_price'] = df['price'].apply(lambda price: np.log(price + 1))
        # drop the 'price'
        df = df.drop(columns=['price'])
        return df

    
    def get_address(self, addr: str) -> str:
        """ 
        =================================================================
            Extract the address.
        """
        result = addr.split(',')[-2:-1]  # select the city
        result = [x.strip() for x in result]  # remove the white spaces
        result = ', '.join(result)  # join on spaces (no longer a list)
        return result


    def clean_text(self, text: str) -> str:
        """
        ====================================================================
            Clean the text.
        """
        pattern = r"\D+"  # non-digits
        result = re.sub(pattern, '', text, flags=re.I)
        return result

  

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [9]:
df = load_data()
print(len(df))
df.head()

8793


,title,address,bed,bath,toilet,pkn_space,price
7,4 bedroom detached duplex for sale,"Megamound Estate, Lekki County Homes., Ikota, ...",4Bedrooms,5Bathrooms,5Toilets,Save,"75,000,000"
8,4 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",4Bedrooms,4Bathrooms,5Toilets,2Parking Spaces,"47,000,000"
9,5 bedroom semi-detached duplex for sale,"In A Well Secured Estate, Ajah, Lagos",5Bedrooms,4Bathrooms,6Toilets,4Parking Spaces,"80,000,000"
10,3 bedroom detached duplex for sale,"Ikola Road, By Command., Alagbado, Ifako-Ijaiy...",3Bedrooms,Save,NaN,NaN,"27,500,000"
11,3 bedroom terraced duplex for sale,"2nd Tollgate, Lekki, Lagos",3Bedrooms,3Bathrooms,4Toilets,Save,"45,000,000"


In [11]:
pre_process = PreProcessData()
# transform
trans_data = pre_process.transform(df)
trans_data.head()

,location,bed,bath,toilet,pkn_space,log_price
9,0,5,4,6,4.0,18.197537
16,0,5,4,6,4.0,18.197537
23,0,5,4,6,4.0,18.197537
30,0,5,4,6,4.0,18.197537
42,5,4,4,5,4.0,19.209138


In [14]:
X = trans_data.drop(columns=['log_price'])
y = trans_data['log_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [15]:
rf_pipe = Pipeline([
    ('rf', RandomForestRegressor(max_depth=14, n_estimators=160, random_state=123)),
])

rf_pipe

Pipeline(steps=[('rf',
                 RandomForestRegressor(max_depth=14, n_estimators=160,
                                       random_state=123))])

In [16]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('rf',
                 RandomForestRegressor(max_depth=14, n_estimators=160,
                                       random_state=123))])

In [17]:
y_pred = rf_pipe.predict(X_test)

In [18]:
        # address, bed, bath, toilet, pkn_space
new_data = [['Lekki', 5, 5, 5, 4]]
# convert to array
new_data = np.array(new_data)
# encode the data
new_data[:, 0] = le_location.transform(new_data[:, 0])

# make predictions
price = rf_pipe.predict(new_data)
# convert the price from log_price to actual price
act_price = np.exp(price) + 1
act_price = round(act_price[0], 0)  # round to the neaarest Naira

print(f"The estimated cost of the property is NGN {act_price:,}")

NameError: name 'le_location' is not defined

In [19]:
def load_data() -> pd.DataFrame:
    """
    ====================================================
    Load data from a directory and covert into a single dataframe.
    """
    # load the csv files in the 'data' directory and save it in a list
    files = [file for file in os.listdir('data/')]
    all_df = pd.DataFrame()  # empty dataframe
    for file in files:
        df = pd.read_csv(f'data/{file}')   # read each file 
        all_df = pd.concat([all_df, df], axis='index')  # concatenate each file
    # filter out duplicated records
    all_df = all_df[all_df.duplicated(keep='first')]
    return all_df 


class PreProcessData(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
    
    def transform(self, data: pd.DataFrame):
        """
        ====================================================
        Clean and tranform the features.
        """
        
        df = data.copy()
        df = df.drop(columns=['price'])
        ### drop the missing values
        df = df.dropna()
        # clean the data
        for col in df.columns:
            if col == 'address':
                df[col] = df[col].apply(self.get_address)
            elif col != 'title':
                df[col] = df[col].apply(self.clean_text)
                
        # convert to numeric data type
        for col in ['bed', 'bath', 'toilet', 'pkn_space', 'price']:
            df[col] = pd.to_numeric(df[col])

        df = df.drop(columns=['title'])
        # select houses with bedrooms between 2 and 7
        df = df.loc[(df['bed'] > 1) & (df['toilet'] < 8)]
        # select houses with bathrooms between 2 and 7
        df = df.loc[(df['bath'] > 1) & (df['bath'] < 8)]
        # select houses with pkn_space between 1 and 10
        df = df.loc[(df['pkn_space'] > 2) & (df['pkn_space'] < 11)]      
        # fill the missing values in 'pkn_space' with the median value
        median = df['pkn_space'].median()
        df['pkn_space'] = np.where(pd.isna(df['pkn_space']), median, df['pkn_space']) 

        # rename column
        df = df.rename(columns={'address': 'location'}) 
        locations = ['Lekki','Ajah','Ikeja','Ikoyi','Victoria Island (VI)','Ibeju Lekki','Isheri North','Magodo','Maryland',
             'Ikorodu','Surulere','Ojodu','Ipaja','Ikotun','Isolo']

        # Filter out locations with fewer counts
        df = df.loc[df['location'].isin(locations)]
        le_location = LabelEncoder()
        df['location'] = le_location.fit_transform(df['location'])

        return df.to_numpy()

    
    def get_address(self, addr: str) -> str:
        """ 
        =================================================================
            Extract the address.
        """
        result = addr.split(',')[-2:-1]  # select the city
        result = [x.strip() for x in result]  # remove the white spaces
        result = ', '.join(result)  # join on spaces (no longer a list)
        return result


    def clean_text(self, text: str) -> str:
        """
        ====================================================================
            Clean the text.
        """
        pattern = r"\D+"  # non-digits
        result = re.sub(pattern, '', text, flags=re.I)
        return result

  

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [ ]:
X = 

In [16]:
X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [17]:
rf_pipe = Pipeline([
    ('Preprocess', PreProcessData()),
    ('rf', RandomForestRegressor(max_depth=14, n_estimators=160, random_state=123)),
])

rf_pipe

Pipeline(steps=[('Preprocess', PreProcessData()),
                ('rf',
                 RandomForestRegressor(max_depth=14, n_estimators=160,
                                       random_state=123))])

In [20]:
rf_pipe.fit(X_train, y_train)

KeyError: 'price'